In [62]:
pip install kafka-python kafka-python-ng confluent_kafka

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
from confluent_kafka import Producer
from kafka import KafkaProducer, KafkaConsumer
from time import sleep
from json import dumps
import json
import requests
import os

In [6]:
api_key = os.getenv('ALPHA_VANTAGE_API_KEY')
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=INFY&interval=1min&apikey=api_key'
r = requests.get(url)
data = r.json()
print(data)                                      ########## Since this API is providing data on daily basis instead of second by second 
                                                 ########## we are using the downloaded data in method 2

{'Meta Data': {'1. Information': 'Intraday (1min) open, high, low, close prices and volume', '2. Symbol': 'INFY', '3. Last Refreshed': '2024-09-13 19:39:00', '4. Interval': '1min', '5. Output Size': 'Compact', '6. Time Zone': 'US/Eastern'}, 'Time Series (1min)': {'2024-09-13 19:39:00': {'1. open': '23.2400', '2. high': '23.2400', '3. low': '23.2400', '4. close': '23.2400', '5. volume': '10'}, '2024-09-13 19:07:00': {'1. open': '23.2100', '2. high': '23.2100', '3. low': '23.2100', '4. close': '23.2100', '5. volume': '5'}, '2024-09-13 19:00:00': {'1. open': '23.2000', '2. high': '23.2000', '3. low': '23.2000', '4. close': '23.2000', '5. volume': '159376'}, '2024-09-13 18:41:00': {'1. open': '23.1900', '2. high': '23.1900', '3. low': '23.1900', '4. close': '23.1900', '5. volume': '1'}, '2024-09-13 18:38:00': {'1. open': '23.2386', '2. high': '23.2386', '3. low': '23.2386', '4. close': '23.2386', '5. volume': '1'}, '2024-09-13 18:30:00': {'1. open': '23.2000', '2. high': '23.2000', '3. low

In [8]:
##Method-2
from kafka import KafkaProducer, KafkaConsumer

producer = KafkaProducer(bootstrap_servers = ['13.201.85.114:9092'],value_serializer=lambda x:dumps(x).encode('utf-8'))
##producer.send('Stockmarket', value = {'Hello7':'galaxy6'})                                             ######### Testing producer with method 2

In [9]:
Stockdata = pd.read_csv('indexProcessed.csv')

In [10]:
Stockdata.head()

,Index,Date,Open,High,Low,Close,Adj Close,Volume,CloseUSD
0,HSI,1986-12-31,2568.300049,2568.300049,2568.300049,2568.300049,2568.300049,0.0,333.879006
1,HSI,1987-01-02,2540.100098,2540.100098,2540.100098,2540.100098,2540.100098,0.0,330.213013
2,HSI,1987-01-05,2552.399902,2552.399902,2552.399902,2552.399902,2552.399902,0.0,331.811987
3,HSI,1987-01-06,2583.899902,2583.899902,2583.899902,2583.899902,2583.899902,0.0,335.906987
4,HSI,1987-01-07,2607.100098,2607.100098,2607.100098,2607.100098,2607.100098,0.0,338.923013


In [11]:
while True:
    dict_stock = Stockdata.sample(1).to_dict(orient="records")[0]
    producer.send('Stockmarket', value=dict_stock)
    sleep(1)                                                            ############### Below error occurs when we stop the producer

KeyboardInterrupt: 

In [11]:
producer.flush()  ########## Flush the producer